In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.rcParams['savefig.dpi'] = 1.5 * matplotlib.rcParams['savefig.dpi']

import numpy as np
import libstempo as T
import libstempo.toasim as LT
import os,sys,glob,math
import h5py as h5
import NX01_psr

try:
    from IPython.core.display import clear_output
    have_ipython = True
except ImportError:
    have_ipython = False

day = 24 * 3600
year = 365.25 * day

In [2]:
# name, hdf5-path, par-path, tim-path
psr_pathinfo = np.genfromtxt('../data/psrList_sim2.txt', dtype=str, skip_header=2)

tmp_psr = []
for ii,tmp_name in enumerate(psr_pathinfo[0:10,0],start=0):
    tmp_psr.append(h5.File(psr_pathinfo[ii,1], 'r')[tmp_name])
                                                                                                                     
psr = [NX01_psr.PsrObjFromH5(p) for p in tmp_psr]

# Grab all the pulsar quantities
[p.grab_all_vars() for p in psr]

# simulation noise properties
pwhite = np.array([1.58,2.60,1.47,1.99,1.65,0.26,0.65,1.51,0.12,1.19])
predAmp = np.array([-13.90,-14.14,-13.09,-13.24,-18.56,-14.9,-13.6,-16.0,-13.99,-13.87])
predGam = np.array([3.18,2.58,1.65,0.03,4.04,4.85,2.00,1.35,2.06,4.02])

# reading in sim2 par files
parfiles = sorted(glob.glob('../data/sim_parfiles/*_stripped.par'))

for ii in range(len(psr)):
    print psr[ii].name, parfiles[ii], pwhite[ii], predAmp[ii], predGam[ii]

--> Extracting J0613-0200 from hdf5 file
--> Done extracting pulsar from hdf5 file :-) 

--> Extracting J0751+1807 from hdf5 file
--> Done extracting pulsar from hdf5 file :-) 

--> Extracting J1012+5307 from hdf5 file
--> Done extracting pulsar from hdf5 file :-) 

--> Extracting J1640+2224 from hdf5 file
--> Done extracting pulsar from hdf5 file :-) 

--> Extracting J1643-1224 from hdf5 file
--> Done extracting pulsar from hdf5 file :-) 

--> Extracting J1713+0747 from hdf5 file
--> Done extracting pulsar from hdf5 file :-) 

--> Extracting J1744-1134 from hdf5 file
--> Done extracting pulsar from hdf5 file :-) 

--> Extracting J1857+0943 from hdf5 file
--> Done extracting pulsar from hdf5 file :-) 

--> Extracting J1909-3744 from hdf5 file
--> Done extracting pulsar from hdf5 file :-) 

--> Extracting J2145-0750 from hdf5 file
--> Done extracting pulsar from hdf5 file :-) 

J0613-0200 ../data/sim_parfiles/J0613-0200_stripped.par 1.58 -13.9 3.18
J0751+1807 ../data/sim_parfiles/J0751+

From here on we iterate over the number of datasets we want to construct.

In [3]:
dataset = '../data/simulated_data/dataset_test'
Agwb = 5e-15
if not os.path.exists(dataset):
    os.makedirs(dataset)

In [4]:
psrtmp = []
for ii in range(len(psr)):

    psrtmp.append(LT.fakepulsar(parfile = parfiles[ii],
                                obstimes = psr[ii].toas,  
                                toaerr = psr[ii].toaerrs/1e-6))

## Add in pulsar white noise

for ii in range(len(psrtmp)):
    LT.add_efac(psrtmp[ii], efac=1.0)

for ii in range(len(psrtmp)):
    psrtmp[ii].fit(iters=5)

## Add in red noise with same spectral properties as Sim2

for ii in range(len(psrtmp)):
    LT.add_rednoise(psrtmp[ii],10.0**predAmp[ii],predGam[ii],
                    components=50,seed=None)

for ii in range(len(psrtmp)):
    psrtmp[ii].fit(iters=5)
    
## Add in GWB 
LT.createGWB(psrtmp, Agwb, 13./3., seed=None)

for ii in range(len(psrtmp)):
    psrtmp[ii].fit(iters=5)

## Save par and tim files
for ii in range(len(psrtmp)):
    psrtmp[ii].savepar(dataset+'/{0}_optstatsim.par'.format(psrtmp[ii].name))
    psrtmp[ii].savetim(dataset+'/{0}_optstatsim.tim'.format(psrtmp[ii].name))

Here we run the individual noise analyses for each pulsar.

In [8]:
for psr in psrtmp:
    print 'Running noise analysis for {0}\n'.format(psr.name)
    
    directory = dataset + '/noiseanalysis/{0}/'.format(psr.name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    pfile = dataset + '/{0}_optstatsim.par'.format(psr.name)
    tfile = dataset + '/{0}_optstatsim.tim'.format(psr.name)
    
    os.system('python NX01_master.py \
                --parfile={0} --timfile={1} \
                --noEcorr --nmodes=20 --redSpecModel=powerlaw --redPrior=loguniform \
                --sampler=ptmcmc \
                --dirExt={2}'.format(pfile, tfile, directory))

Running noise analysis for J0613-0200

Running noise analysis for J0751+1807

Running noise analysis for J1012+5307

Running noise analysis for J1640+2224

Running noise analysis for J1643-1224

Running noise analysis for J1713+0747

Running noise analysis for J1744-1134

Running noise analysis for J1857+0943

Running noise analysis for J1909-3744

Running noise analysis for J2145-0750



In [7]:
os.path.exists('./optstatsim_results/')

False